In [1]:
import pandas as pd
import numpy as np
import pickle

In [12]:
df = pd.read_csv("./waqi-covid19-airqualitydata-2020.csv", skiprows=4)
days = 30

In [57]:
df = df[df["Country"] == 'IN']
df = df[["Date","City","Specie","max"]]
df.replace("wind speed", "wind-speed", inplace=True)
df.replace("wind gust", "wind-gust", inplace=True)
print(df.shape)
df.head()

(63934, 4)


,Date,City,Specie,max
587494,2021-03-02,Thrissur,dew,25.0
587495,2021-03-17,Thrissur,dew,25.0
587496,2021-07-28,Thrissur,dew,24.5
587497,2021-09-02,Thrissur,dew,25.0
587498,2021-02-17,Thrissur,dew,24.0


In [3]:
cities = 18 #len(df.City.unique())

In [59]:
df.Specie.unique()

array(['dew', 'pm10', 'no2', 'so2', 'pm25', 'pressure', 'wind-speed',
       'co', 'temperature', 'humidity', 'o3', 'wind-gust',
       'precipitation', 'wd'], dtype=object)

In [60]:
dates = []
date_vals = []
for d in df.Date.unique():
    if d[5:7] in ["04"]: dates.append(d)
for d in dates:
    date_vals.append(int(d[5:7])*100 + int(d[8:]))
#date_vals.append(414)
date_vals.sort()
date_vals = date_vals[:days]
print(date_vals)

[401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430]


In [61]:
for i in range(df.shape[0]):
    d = df.iloc[i,0]
    d = int(d[5:7])*100 + int(d[8:])
    df.iloc[i,0] = d
df.head()

,Date,City,Specie,max
587494,302,Thrissur,dew,25.0
587495,317,Thrissur,dew,25.0
587496,728,Thrissur,dew,24.5
587497,902,Thrissur,dew,25.0
587498,217,Thrissur,dew,24.0


In [13]:
features = ["Date"] + list(df.Specie.unique())

In [63]:
cls = dict()
gen = pd.DataFrame(np.zeros((days,15)), columns= features)
gen.replace(0,None, inplace=True)
for c in df.City.unique():
    cls[c] = gen.copy()

In [64]:
df.reset_index(inplace=True)

In [65]:
for i in range(df.shape[0]):
    if df["Date"][i] in date_vals:
        r = date_vals.index(df["Date"][i])
        c = features.index(df["Specie"][i])
        cls[df["City"][i]].iloc[r, 0] = df["Date"][i]
        cls[df["City"][i]].iloc[r, c] = df["max"][i]

In [66]:
cls["Thrissur"].head()

,Date,dew,pm10,no2,so2,pm25,pressure,wind-speed,co,temperature,humidity,o3,wind-gust,precipitation,wd
0,401.0,26.5,97.0,5.1,64.6,151.0,1009.0,5.9,10.8,34.0,98.9,29.8,0.0,0.0,0.0
1,402.0,24.0,73.0,5.1,15.0,189.0,1009.0,5.4,9.9,32.5,98.9,34.6,0.0,0.0,0.0
2,403.0,25.0,108.0,6.9,9.2,149.0,1010.5,6.1,10.1,33.0,97.9,20.7,0.0,0.0,0.0
3,404.0,24.5,85.0,5.2,11.4,161.0,1011.0,5.1,9.2,33.0,98.5,20.6,0.0,0.0,0.0
4,405.0,26.0,96.0,5.2,490.0,152.0,1011.0,5.4,9.1,34.5,88.8,20.6,0.0,0.0,0.0


In [67]:
final = pd.DataFrame(np.zeros((days*cities,16)), columns=["City"] + features)
for i in range(cities):
    final.iloc[i*days:i*days + days,0] = list(cls.keys())[i]
    final.iloc[i*days:i*days + days,1:] = cls[list(cls.keys())[i]].iloc[:,:]
final.head()

,City,Date,dew,pm10,no2,so2,pm25,pressure,wind-speed,co,temperature,humidity,o3,wind-gust,precipitation,wd
0,Thrissur,401.0,26.5,97.0,5.1,64.6,151.0,1009.0,5.9,10.8,34.0,98.9,29.8,0.0,0.0,0.0
1,Thrissur,402.0,24.0,73.0,5.1,15.0,189.0,1009.0,5.4,9.9,32.5,98.9,34.6,0.0,0.0,0.0
2,Thrissur,403.0,25.0,108.0,6.9,9.2,149.0,1010.5,6.1,10.1,33.0,97.9,20.7,0.0,0.0,0.0
3,Thrissur,404.0,24.5,85.0,5.2,11.4,161.0,1011.0,5.1,9.2,33.0,98.5,20.6,0.0,0.0,0.0
4,Thrissur,405.0,26.0,96.0,5.2,490.0,152.0,1011.0,5.4,9.1,34.5,88.8,20.6,0.0,0.0,0.0


In [68]:
final.to_csv("./2021_april_clean.csv", index=False)

In [54]:
with open('2021_april.pickle', 'wb') as handle:
    pickle.dump(cls, handle, protocol=pickle.HIGHEST_PROTOCOL)
handle.close()

In [5]:
with open('2019_april.pickle', 'rb') as handle:
    cls = pickle.load(handle)
handle.close()


In [6]:
req_cities = ['Hyderabad', 'Delhi', 'Chandigarh', 'Bhopal', 'Lucknow', 'Ghāziābād', 'Hāpur', 'Gandhinagar', 'Chennai', 'Nashik', 'Visakhapatnam', 'Mumbai', 'Jaipur', 'Patna', 'Muzaffarnagar', 'Thiruvananthapuram', 'Bengaluru', 'Kolkata']

In [22]:
features = ["City", "Date"] + ['humidity', 'temperature', 'co', 'pm10', 'dew', 'pm25', 'pressure', 'o3', 'so2', 'no2', 'wind-speed',"wd","wind-gust","precipitation"]

In [36]:
def remove_cities(file):
    with open(file, 'rb') as handle:
        cls = pickle.load(handle)
    handle.close()

    kay = list(cls.keys())

    for c in kay:
        if c not in req_cities:
            cls.pop(c)
    
    days = 30
    cities = len(req_cities)

    final = pd.DataFrame(np.zeros((days*cities,16)), columns=["City"] + list(cls[c].columns))
    for i in range(cities):
        final.iloc[i*days:i*days + days,0] = list(cls.keys())[i]
        final.iloc[i*days:i*days + days,1:] = cls[list(cls.keys())[i]].iloc[:,:]
    return final

In [39]:
df19 = remove_cities("./2019_april.pickle")

In [40]:
df19.to_csv("./2019_april.csv", index=False)